<a href="https://colab.research.google.com/github/AaryanRankhamb/ML-Projects/blob/main/SentimentAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
raw_train, raw_valid, raw_test = tfds.load(
    name="imdb_reviews",
    split=["train[:90%]", "test[90%:]", "test"],
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHZYJFG/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHZYJFG/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHZYJFG/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
import numpy as np
for reviews, labels in raw_train.take(4):
  print(reviews.numpy().decode('utf-8'))
  print(labels.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was co

In [ ]:
tf.random.set_seed(42)
train_ds = raw_train.shuffle(buffer_size=5000).batch(32).prefetch(1)
test_ds = raw_test.batch(32).prefetch(1)
valid_ds = raw_valid.batch(32).prefetch(1)

In [ ]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_ds.map(lambda review, label: review))


In [ ]:
embed_size = 128
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_size, mask_zero=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(
    optimizer = 'nadam',
    loss = "binary_crossentropy",
    metrics=['accuracy']
)

cbk = tf.keras.callbacks.ModelCheckpoint("my_sentiment_analyzer", save_best_only=True, monitor="val_accuracy")

In [ ]:
model.fit(train_ds, epochs=2, validation_data=valid_ds, callbacks=[cbk])

Epoch 1/2
704/704 [==============================] - 529s 745ms/step - loss: 0.5758 - accuracy: 0.6912 - val_loss: 0.5106 - val_accuracy: 0.7428
Epoch 2/2
704/704 [==============================] - 535s 760ms/step - loss: 0.4151 - accuracy: 0.8157 - val_loss: 0.3581 - val_accuracy: 0.8508


In [ ]:
model.evaluate(test_ds)

782/782 [==============================] - 98s 125ms/step - loss: 0.3417 - accuracy: 0.8554


[0.3416970670223236, 0.855400025844574]

In [ ]:
review = '''One of the most anticipated films of the year for many people, myself included, Oppenheimer largely delivers. Much of it's great. I feel like I loved two of its three hours, and liked the other hour.... but it's that fact that stops me from adoring the entire thing. I know with Christopher Nolan's Dunkirk, that clicked on a second watch, so maybe Oppenheimer will need one too. That being said, I don't feel the need to rush out and see it again too soon, because it was a long and exhausting film.
'''

model.predict([review])

1/1 [==============================] - 0s 33ms/step


array([[0.9360397]], dtype=float32)

In [ ]:
model.save("my_movie_review_sentiment_analyzer", save_format = "tf")